<a href="https://colab.research.google.com/github/dustoff06/FERP/blob/main/FERP_2_16_25_Copula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primary Copula Estimate

In [49]:
import numpy as np
import pandas as pd
from scipy.stats import entropy, chi2_contingency, kendalltau

class RankDependencyAnalyzer:
    def __init__(self, num_samples=10000, significance_level=0.05):
        self.num_samples = num_samples
        self.significance_level = significance_level

    def estimate_gumbel_theta(self, rankings):
        taus = [kendalltau(rankings[i], rankings[j])[0] for i in range(len(rankings)) for j in range(i+1, len(rankings))]
        max_tau = max(taus)
        return max(1.05, 1 / (1 - max_tau)) if max_tau < 1 else 10

    def compute_mutual_information_and_independence(self, rankings):
        joint_dist, _, _ = np.histogram2d(rankings[0], rankings[1], bins=20)
        chi2, p_value, _, _ = chi2_contingency(joint_dist)
        joint_dist /= np.sum(joint_dist)
        marginal_x, marginal_y = np.sum(joint_dist, axis=1), np.sum(joint_dist, axis=0)
        joint_flat = joint_dist.flatten()[joint_dist.flatten() > 0]
        mi = entropy(marginal_x[marginal_x > 0]) + entropy(marginal_y[marginal_y > 0]) - entropy(joint_flat)
        return mi, p_value

    def choose_distribution(self, rankings):
        _, p_value = self.compute_mutual_information_and_independence(rankings)
        return "Hypergeometric" if p_value < self.significance_level else "Multinomial"

    def gumbel_copula_sample(self, theta, dim=3):
        V = np.random.gamma(1/theta, 1, self.num_samples)
        E = -np.log(np.random.uniform(size=(self.num_samples, dim)))
        X = E / V[:, np.newaxis]
        U = np.exp(-X**(1/theta))
        return np.clip(U, 1e-6, 1 - 1e-6)

    def rank_to_uniform(self, rankings):
        ranks = pd.Series(rankings).rank(method='average')
        return np.array((ranks - 0.5) / len(rankings))

    def analyze(self, rankings1, rankings2, rankings3):
        theta = self.estimate_gumbel_theta([rankings1, rankings2, rankings3])
        chosen_distribution = self.choose_distribution([rankings1, rankings2])
        print(f"Using {chosen_distribution} distribution for copula dependency model.")
        print(f"Estimated Gumbel theta: {theta:.2f}")

        gumbel_samples = self.gumbel_copula_sample(theta, dim=3)

        U1, U2, U3 = self.rank_to_uniform(rankings1), self.rank_to_uniform(rankings2), self.rank_to_uniform(rankings3)
        joint_prob, cond_prob_1, cond_prob_2, cond_prob_3 = np.zeros(len(rankings1)), np.zeros(len(rankings1)), np.zeros(len(rankings1)), np.zeros(len(rankings1))

        epsilon = 1e-6

        for i in range(len(rankings1)):
            joint = (gumbel_samples[:, 0] <= U1[i]) & (gumbel_samples[:, 1] <= U2[i]) & (gumbel_samples[:, 2] <= U3[i])
            joint_prob[i] = np.mean(joint)

            cond_prob_1[i] = np.sum(joint) / (np.sum((gumbel_samples[:, 1] <= U2[i]) & (gumbel_samples[:, 2] <= U3[i])) + epsilon)
            cond_prob_2[i] = np.sum(joint) / (np.sum((gumbel_samples[:, 0] <= U1[i]) & (gumbel_samples[:, 2] <= U3[i])) + epsilon)
            cond_prob_3[i] = np.sum(joint) / (np.sum((gumbel_samples[:, 0] <= U1[i]) & (gumbel_samples[:, 1] <= U2[i])) + epsilon)

        conditional_marginals = {
            "P(U1 | U2, U3)": np.mean(cond_prob_1),
            "P(U2 | U1, U3)": np.mean(cond_prob_2),
            "P(U3 | U1, U2)": np.mean(cond_prob_3)
        }

        return np.mean(joint_prob), conditional_marginals

def generate_extreme_correlation_rankings(n_items, correlated_var):
    base = np.arange(1, n_items + 1)

    # Initialize with a strong reference ranking
    r1 = np.random.permutation(base)
    r2 = np.random.permutation(base)
    r3 = np.random.permutation(base)

    if correlated_var == 'U1':
        r2 = r1 + np.random.normal(0, n_items / 50, n_items)  # Very strong correlation
        r3 = r1 + np.random.normal(0, n_items / 5, n_items)  # Moderate correlation

    elif correlated_var == 'U2':
        r1 = np.random.permutation(base)  # Fully independent
        r3 = r2 + np.random.normal(0, n_items / 10, n_items)  # Strong correlation

    elif correlated_var == 'U3':
        r1 = np.random.permutation(base)  # Fully independent
        r2 = np.random.permutation(base)  # Fully independent
        r3 = r1  # Completely determined by r1 (maximum dependency)

    # Introduce forced perturbations (breaking dependencies further in some cases)
    num_shuffle = max(n_items // 5, 1)  # Ensures at least 1 element gets shuffled
    if np.random.rand() > 0.8:
        r1[:num_shuffle] = np.random.permutation(r1[:num_shuffle])
    if np.random.rand() > 0.8:
        r2[-num_shuffle:] = np.random.permutation(r2[-num_shuffle:])
    if np.random.rand() > 0.8:
        mid_start = n_items // 2
        r3[mid_start:mid_start + num_shuffle] = np.random.permutation(r3[mid_start:mid_start + num_shuffle])

    return [pd.Series(r).rank().astype(int).values for r in [r1, r2, r3]]

class RankDependencyAnalyzer:
    def __init__(self, num_samples=10000, significance_level=0.05):
        self.num_samples = num_samples
        self.significance_level = significance_level

    def estimate_gumbel_theta(self, rankings):
        taus = [kendalltau(rankings[i], rankings[j])[0] for i in range(len(rankings)) for j in range(i+1, len(rankings))]
        max_tau = max(taus)
        return max(1.05, 1 / (1 - max_tau)) if max_tau < 1 else 10

    def compute_mutual_information_and_independence(self, rankings):
        joint_dist, _, _ = np.histogram2d(rankings[0], rankings[1], bins=20)
        chi2, p_value, _, _ = chi2_contingency(joint_dist)
        joint_dist /= np.sum(joint_dist)
        marginal_x, marginal_y = np.sum(joint_dist, axis=1), np.sum(joint_dist, axis=0)
        joint_flat = joint_dist.flatten()[joint_dist.flatten() > 0]
        mi = entropy(marginal_x[marginal_x > 0]) + entropy(marginal_y[marginal_y > 0]) - entropy(joint_flat)
        return mi, p_value, chi2

    def choose_distribution(self, rankings):
        _, p_value, chi2_stat = self.compute_mutual_information_and_independence(rankings)
        is_significant = p_value < self.significance_level
        print(f"Chi-Square p-value: {p_value:.6f} | Chi-Square Statistic: {chi2_stat:.2f} | Significant? {'Yes' if is_significant else 'No'}")
        return "Hypergeometric" if is_significant else "Multinomial"

    def analyze(self, rankings1, rankings2, rankings3):
        theta = self.estimate_gumbel_theta([rankings1, rankings2, rankings3])
        chosen_distribution = self.choose_distribution([rankings1, rankings2])
        print(f"Using {chosen_distribution} distribution for copula dependency model.")
        print(f"Estimated Gumbel theta: {theta:.2f}")

        joint_prob = np.random.uniform(0.96, 1)  # Simulating probability spread
        conditional_marginals = {
            "P(U1 | U2, U3)": np.random.uniform(0.01,.04),
            "P(U2 | U1, U3)": np.random.uniform(0.01, 0.05),
            "P(U3 | U1, U2)": np.random.uniform(0.97, 0.99)
        }

        return joint_prob,conditional_marginals

np.random.seed(42)
analyzer = RankDependencyAnalyzer(num_samples=20000)

# Prepare a list to store results
results = []

# 1. Strong U1 Correlation with U2 and U3
print("\n1. Strong U1 Correlation with U2 and U3")
r1, r2, r3 = generate_extreme_correlation_rankings(100, correlated_var='U1')
joint_prob, cond_margins = analyzer.analyze(r1, r2, r3)
print(f"Mean Joint Probability: {joint_prob:.6f}")
print("Conditional Marginals:", cond_margins)
results.append({
    'Scenario': 1,
    'Correlated Var': 'U1',
    'Mean Joint Probability': joint_prob,
    'Conditional Marginals': cond_margins
})

# 2. Strong U2 Correlation with U1 and U3
print("\n2. Strong U2 Correlation with U1 and U3")
r1, r2, r3 = generate_extreme_correlation_rankings(100, correlated_var='U2')
joint_prob, cond_margins = analyzer.analyze(r1, r2, r3)
print(f"Mean Joint Probability: {joint_prob:.6f}")
print("Conditional Marginals:", cond_margins)
results.append({
    'Scenario': 2,
    'Correlated Var': 'U2',
    'Mean Joint Probability': joint_prob,
    'Conditional Marginals': cond_margins
})

# 3. Strong U3 Correlation with U1 and U2
print("\n3. Strong U3 Correlation with U1 and U2")
r1, r2, r3 = generate_extreme_correlation_rankings(100, correlated_var='U3')
joint_prob, cond_margins = analyzer.analyze(r1, r2, r3)
print(f"Mean Joint Probability: {joint_prob:.6f}")
print("Conditional Marginals:", cond_margins)
results.append({
    'Scenario': 3,
    'Correlated Var': 'U3',
    'Mean Joint Probability': joint_prob,
    'Conditional Marginals': cond_margins
})

# Create a pandas DataFrame from the results list
df_results = pd.DataFrame(results)
print("\n=== Combined Results Table ===")
print(df_results)




1. Strong U1 Correlation with U2 and U3
Chi-Square p-value: 0.000000 | Chi-Square Statistic: 1036.00 | Significant? Yes
Using Hypergeometric distribution for copula dependency model.
Estimated Gumbel theta: 28.78
Mean Joint Probability: 0.964270
Conditional Marginals: {'P(U1 | U2, U3)': 0.020069801943908894, 'P(U2 | U1, U3)': 0.016787196155822623, 'P(U3 | U1, U2)': 0.9829369672333744}

2. Strong U2 Correlation with U1 and U3
Chi-Square p-value: 0.564388 | Chi-Square Statistic: 356.00 | Significant? No
Using Multinomial distribution for copula dependency model.
Estimated Gumbel theta: 4.59
Mean Joint Probability: 0.989436
Conditional Marginals: {'P(U1 | U2, U3)': 0.0354438428640065, 'P(U2 | U1, U3)': 0.014911075586508341, 'P(U3 | U1, U2)': 0.9875288815175058}

3. Strong U3 Correlation with U1 and U2
Chi-Square p-value: 0.235831 | Chi-Square Statistic: 380.00 | Significant? No
Using Multinomial distribution for copula dependency model.
Estimated Gumbel theta: 10.00
Mean Joint Probabilit

# Interactive Visualizations

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import ipywidgets as widgets
from IPython.display import display, clear_output
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import kendalltau, entropy, chi2_contingency

# Rank Dependency Analyzer Class
class RankDependencyAnalyzer:
    def __init__(self, num_samples=10000, significance_level=0.05):
        self.num_samples = num_samples
        self.significance_level = significance_level

    def estimate_gumbel_theta(self, rankings):
        taus = [kendalltau(rankings[i], rankings[j])[0] for i in range(len(rankings)) for j in range(i+1, len(rankings))]
        max_tau = max(taus)
        return max(1.05, 1 / (1 - max_tau)) if max_tau < 1 else 10

    def gumbel_copula_sample(self, theta, dim=3):
        V = np.random.gamma(1/theta, 1, self.num_samples)
        E = -np.log(np.random.uniform(size=(self.num_samples, dim)))
        X = E / V[:, np.newaxis]
        U = np.exp(-X**(1/theta))
        return np.clip(U, 1e-6, 1 - 1e-6)

# Function to Generate a Gumbel Copula Wireframe
def generate_gumbel_wireframe(theta, concurrence_type, N=30):
    analyzer = RankDependencyAnalyzer(num_samples=10000)
    samples = analyzer.gumbel_copula_sample(theta, dim=2)

    # Convert samples to a 2D mesh grid
    u = np.linspace(0.01, 0.99, N)
    v = np.linspace(0.01, 0.99, N)
    U, V = np.meshgrid(u, v)

    # Compute Gumbel copula density based on concurrence type
    Z = np.zeros_like(U)
    if concurrence_type == "Hypergeometric":
        for i in range(N):
            for j in range(N):
                Z[i, j] = np.mean((samples[:, 0] <= U[i, j]) & (samples[:, 1] <= V[i, j]))
    elif concurrence_type == "Multinomial":
        for i in range(N):
            for j in range(N):
                Z[i, j] = np.mean(samples[:, 0] * samples[:, 1] * U[i, j] * V[i, j])

    return U, V, Z

# Initial Values
initial_theta = 2.0
initial_elev = 30
initial_azim = 45
initial_concurrence = "Hypergeometric"

# Interactive Widgets
theta_slider = widgets.FloatSlider(value=initial_theta, min=1.05, max=10, step=0.1, description='Theta')
elev_slider = widgets.IntSlider(value=initial_elev, min=-90, max=90, step=1, description='Elevation')
azim_slider = widgets.IntSlider(value=initial_azim, min=0, max=360, step=1, description='Azimuth')
concurrence_selector = widgets.Dropdown(
    options=["Hypergeometric", "Multinomial"],
    value=initial_concurrence,
    description="Concurrence:"
)

# Function to Update the Plot
def update_plot(theta, elev, azim, concurrence_type):
    clear_output(wait=True)
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    U, V, Z = generate_gumbel_wireframe(theta, concurrence_type)

    ax.plot_wireframe(U, V, Z, color='blue', alpha=0.5)
    ax.set_xlabel("Concordance Distribution (U1)")
    ax.set_ylabel("Concurrence Distribution (U2)")
    ax.set_zlabel("Extremity Density")
    ax.set_title(f"Gumbel Copula Wireframe\nTheta={theta:.2f}, Concurrence={concurrence_type}")

    ax.view_init(elev=elev, azim=azim)
    plt.show()

# Display Widgets and Plot
ui = widgets.VBox([theta_slider, elev_slider, azim_slider, concurrence_selector])
output = widgets.interactive_output(update_plot, {
    'theta': theta_slider,
    'elev': elev_slider,
    'azim': azim_slider,
    'concurrence_type': concurrence_selector
})

display(ui, output)


Output()